In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


TensorFlow session

In [9]:
sess = tf.InteractiveSession()

Data placeholders

(les données sont empilées selon dimension 0)

In [10]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

Paramètres du graphe (Variables) +° initialisation

In [11]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

Prediction graph

In [12]:
y = tf.matmul(x,W) + b

Loss graph

In [13]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

Optimization graph

In [14]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Training

In [15]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

Evaluation graph

In [16]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Evaluation

In [17]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9195


## Multilayer CNN

Couche de poids paramétrable

In [18]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

Couche de biais paramétrable

In [19]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

Construction d'un operateur de convolution

(la taille de la fenetre de convolution est donnée par W)

In [20]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

Contruction d'un opérateur de max pooling

(la dim 0 correspond au batch size, la dim 4 à la profondeur si canaux multiples) [1, 2, 2, 1] <--> [batch, i, j, depth]

In [21]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## Prediction graph

Construction couche 1

In [22]:
# Paramètres
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
# Input 2D reshape
x_image = tf.reshape(x, [-1, 28, 28, 1])
# Graph construction
z_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
z_pool1 = max_pool_2x2(z_conv1)

Construction couche 2

In [23]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

z_conv2 = tf.nn.relu(conv2d(z_pool1, W_conv2) + b_conv2)
z_pool2 = max_pool_2x2(z_conv2)

Construction couche 3

In [24]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

z_pool2_flat = tf.reshape(z_pool2, [-1, 7*7*64])
z_fc1 = tf.nn.relu(tf.matmul(z_pool2_flat, W_fc1) + b_fc1)

Dropout graph

In [25]:
keep_prob = tf.placeholder(tf.float32)
z_fc1_drop = tf.nn.dropout(z_fc1, keep_prob)

Readout graph

In [26]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_cnn = tf.matmul(z_fc1_drop, W_fc2) + b_fc2

## Loss graph

In [27]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_cnn))

## Train graph

In [28]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## Evaluation graph

In [29]:
correct_prediction = tf.equal(tf.argmax(y_cnn, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training

In [31]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 10 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            sys.stdout.write('\rstep %d, training accuracy %g' % (i, train_accuracy))
            sys.stdout.flush()
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 2530, training accuracy 0.98

KeyboardInterrupt: 